In [ ]:
from google.colab import drive # mounts the google drive for a new notebook 
print('Mounting Drive ...')
drive.mount('/content/drive')

In [ ]:
# load the 2 npy files created by the process_yale_images.ipynb 
from numpy import load
import numpy as np
path = '/content/drive/My Drive/Assignment II/'
# load array
print("Reading Data ...")
X = load(path + 'Copy of yaleExtB_data.npy')
print('Data Reading Completed.')
print('Reading Target Data ...')
target = load(path + 'Copy of yaleExtB_target.npy')
print("Target Data Reading Completed.")

# Using TensorFlow For Meta Learning

In [ ]:
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder(sparse_output=False)
reshaped = target.reshape(len(target), 1)
Y = onehot_encoder.fit_transform(reshaped)

In [ ]:
# splitting training and testing data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.33, random_state=26)

In [ ]:
# calculating pca
from sklearn.decomposition import PCA # Used to perform Principal Component Analysis

# Defining the number of principal components to use for PCA.
nof_prin_components = 200

# Initializing a PCA object with the desired number of principal components and fits it to the training data.
pca = PCA(n_components=nof_prin_components, whiten=True).fit(X_train)

# Transforming the training data into the new feature space defined by the principal components.
X_train_pca = pca.transform(X_train)

# Transforming the testing data into the new feature space defined by the principal components.
X_test_pca = pca.transform(X_test)

# Trasforming the whole dataset into the new feature space defined bt the principal components.
X_pca = pca.transform(X)

In [ ]:
# configuring a model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(30, activation='softmax')
])

model.compile(optimizer ='SGD', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_pca, y_train, batch_size=5, epochs=15)

In [ ]:
# Testing Model
loss, accuracy = model.evaluate(X_test_pca, y_test, verbose=0)

print('Test Loss : ', loss)
print('Test accuracy : ', accuracy)

In [ ]:
y_pred = model.predict(X_test_pca)

In [ ]:
import numpy as np

actual = np.argmax(y_test, axis=1)
predicted = np.argmax(y_pred, axis=1)

print(f'Actual : {actual}')
print("Predicted : ", predicted)

## Adaptive Boosting

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

# splitting training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size = 0.33, random_state=26)

In [ ]:
# calculating pca
from sklearn.decomposition import PCA # Used to perform Principal Component Analysis

# Defining the number of principal components to use for PCA.
nof_prin_components = 200

# Initializing a PCA object with the desired number of principal components and fits it to the training data.
pca = PCA(n_components=nof_prin_components, whiten=True).fit(X_train)

# Transforming the training data into the new feature space defined by the principal components.
X_train_pca = pca.transform(X_train)

# Transforming the testing data into the new feature space defined by the principal components.
X_test_pca = pca.transform(X_test)

# Trasforming the whole dataset into the new feature space defined bt the principal components.
X_pca = pca.transform(X)

In [ ]:
parameters = {
    'n_estimators': [200, 300, 400],
    'learning_rate': [0.01, 0.1, 0.5],
    'algorithm': ['SAMME', 'SAMME.R']
}

adaboost = AdaBoostClassifier()

### Grid Search in AdaBoosting

In [ ]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(adaboost, param_grid=parameters, cv=5)
grid_search.fit(X_train, y_train)

# Get the best model and its hyperparameters
best_model = grid_search.best_estimator_
best_para = grid_search.best_params_

# Evaluate the best model on the test data
test_accuracy = best_model.score(X_test, y_test)

print("Best Model : ", best_model)
print("Best Parameter", best_para)
print('Accuracy : ', test_accuracy)

### Random Search in AdaBooosting

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

random_search = RandomizedSearchCV(adaboost, param_distributions=parameters, cv=5)
random_search.fit(X_train, y_train)

# Get the best model and its hyperparameters
best_model = random_search.best_estimator_
best_para = random_search.best_params_

# Evaluate the best model on the test data
test_accuracy = best_model.score(X_test, y_test)

print("Best Model : ", best_model)
print("Best Parameter", best_para)
print('Accuracy : ', test_accuracy)